In [1]:
#!pip install polars

# import

In [2]:
import gc
import os
import json
import sys

sys.path.append("/kaggle/input/sentence-transformers-v222/sentence-transformers-master")

import cupy as cp
import numpy as np
import pandas as pd
#import pytorch_lightning as pl
#import polars as pl
import sklearn.metrics as metrics
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from cuml.metrics import pairwise_distances
from cuml.neighbors import NearestNeighbors
from pytorch_lightning import Trainer
from pytorch_lightning import seed_everything
from pytorch_lightning.loggers import WandbLogger
from sklearn.model_selection import StratifiedGroupKFold
from transformers import (
    AutoConfig,
    AutoModel,
    AutoTokenizer,
)

from tqdm import tqdm
tqdm.pandas()

from sentence_transformers import SentenceTransformer, LoggingHandler
from sentence_transformers import models, util, datasets, evaluation, losses

# Config

In [3]:
cfg = {
    "general": {
        "project_name": "LECR",
        "input_path": "/kaggle/input",
        "seed": 42,
        "cv": True,
        #"wandb_desabled": False,
        "fold": [0, 1, 2, 3], # list (0-idx start) or null. set one element list, hold-out mode.
        #"n_splits": 15,
    },
    
    "task": {
        "select_top_n": 10,
        "metric": "cosine",
    },
    
    #"model": {
    #    "model_name_other": "/kaggle/input/lecr-retriever-model/alllang",
    #    "model_name_en": "/kaggle/input/lecr-retriever-model/english",
    #    "model_name_es": "/kaggle/input/lecr-retriever-model/spanish",
    #},
}

# Function

In [4]:
def f2_score(y_true, y_pred):
    y_true = y_true.apply(lambda x: set(x.split()))
    y_pred = y_pred.apply(lambda x: set(x.split()))
    tp = np.array([len(x[0] & x[1]) for x in zip(y_true, y_pred)])
    fp = np.array([len(x[1] - x[0]) for x in zip(y_true, y_pred)])
    fn = np.array([len(x[0] - x[1]) for x in zip(y_true, y_pred)])
    precision = tp / (tp + fp)
    recall = tp / (tp + fn)
    f2 = tp / (tp + 0.2 * fp + 0.8 * fn)
    return round(f2.mean(), 4)

def get_pos_score(y_true, y_pred):
    y_true = y_true.apply(lambda x: set(x.split()))
    y_pred = y_pred.apply(lambda x: set(x.split()))
    int_true = np.array([len(x[0] & x[1]) / len(x[0]) for x in zip(y_true, y_pred)])
    return round(np.mean(int_true), 5)

In [5]:
def build_training_set(topic, content):
    # Create lists for training
    topic_ids = []
    content_ids = []
    topic_inputs = []
    content_inputs = []
    language = []
    targets = []
    dists = []
    # Iterate over each topic
    for k in tqdm(range(len(topic))):
        row = topic.iloc[k]
        topic_id = row["id"]
        topic_input = row["topic_inputs"]
        lang = row["language"]
        predictions = row["predicts"].split(" ")
        ground_truth = row["content_ids"].split(" ")
        dist = row["dists"]
        if predictions[0] == "":
            #topic_ids.append(topic_id)
            #content_ids.append(np.nan)
            #topic_inputs.append(topic_inputs)
            #content_inputs.append(np.nan)
            #targets.append(np.nan)
            #dists.append(np.nan)
            continue
        for i, pred in enumerate(predictions):
            content_input = content.loc[pred, "content_inputs"]
            topic_ids.append(topic_id)
            content_ids.append(pred)
            topic_inputs.append(topic_input)
            content_inputs.append(content_input)
            language.append(lang)
            # If pred is in ground truth, 1 else 0
            if pred in ground_truth:
                targets.append(1)
            else:
                targets.append(0)
            dists.append(dist[i])
    train = pd.DataFrame(
        {"topic_ids": topic_ids, 
         "content_ids": content_ids, 
         "topic_inputs": topic_inputs, 
         "content_inputs": content_inputs, 
         "target": targets,
         "language": language,
         "dists": dists,
        }
    )
    # Release memory
    del topic_ids, content_ids, topic_inputs, content_inputs, targets
    gc.collect()
    return train

# polars
def build_training_set_pl(topic, content):
    # Create lists for training
    topic_ids = []
    content_ids = []
    topic_inputs = []
    content_inputs = []
    language = []
    targets = []
    dists = []
    # Iterate over each topic
    for k in tqdm(range(len(topic))):
        row = topic[k]
        topic_id = row["id"][0]
        topic_input = row["topic_inputs"][0]
        lang = row["language"][0]
        predictions = row["predicts"][0].split(" ")
        ground_truth = row["content_ids"][0].split(" ")
        dist = row["dists"]
        if predictions[0] == "":
            #topic_ids.append(topic_id)
            #content_ids.append(np.nan)
            #topic_inputs.append(topic_inputs)
            #content_inputs.append(np.nan)
            #targets.append(np.nan)
            #dists.append(np.nan)
            continue
        for pred in predictions:
            content_input = content.filter(pl.col("id")==pred)["content_inputs"][0]
            topic_ids.append(topic_id)
            content_ids.append(pred)
            topic_inputs.append(topic_input)
            content_inputs.append(content_input)
            language.append(lang)
            # If pred is in ground truth, 1 else 0
            if pred in ground_truth:
                targets.append(1)
            else:
                targets.append(0)
            dists.append(dist)
    train = pl.DataFrame(
        {"topic_ids": topic_ids, 
         "content_ids": content_ids, 
         "topic_inputs": topic_inputs, 
         "content_inputs": content_inputs, 
         "target": targets,
         "language": language,
         "dists": dists,
        }
    )
    # Release memory
    del topic_ids, content_ids, topic_inputs, content_inputs, targets
    gc.collect()
    return train

## Read Data

In [6]:
def read_data(fold_n, mode="train"):
    content_df = pd.read_pickle(f"{cfg['general']['input_path']}/lecr-datapkl/content.pkl")
    correlations_df = pd.read_csv(f"{cfg['general']['input_path']}/learning-equality-curriculum-recommendations/correlations.csv")
    topics_df = pd.read_pickle(f"{cfg['general']['input_path']}/lecr-datapkl/topics.pkl")
    sub_df = pd.read_csv(f"{cfg['general']['input_path']}/learning-equality-curriculum-recommendations/sample_submission.csv")
    fold = pd.read_pickle(f"{cfg['general']['input_path']}/lecr-datapkl/fold.pkl")
    
    print(f"fold_{fold_n}")
    topics_df_fold = topics_df.merge(fold, how="inner", left_on="id", right_on="topic_id")
    topics_df_fold = topics_df_fold.drop(["id"], axis=1)
    if mode=="train":
        topics_df_fold_indices = topics_df_fold["fold"]!=fold_n
    elif mode=="valid":
        topics_df_fold_indices = topics_df_fold["fold"]==fold_n
    topics_df_fold = topics_df[topics_df_fold_indices]

    topics_df_fold.rename(columns=lambda x: "topic_" + x, inplace=True)
    content_df.rename(columns=lambda x: "content_" + x, inplace=True)

    correlations = correlations_df.copy()
    correlations.content_ids = correlations.content_ids.str.split()
    correlations = correlations.explode("content_ids").rename(columns={"content_ids": "content_id"})
    correlations = correlations.merge(topics_df_fold, how="inner", on="topic_id")
    correlations = correlations.merge(content_df, how="inner", on="content_id")
    correlations.rename(columns={"content_content_inputs" : "content_inputs"}, inplace=True)
    correlations.rename(columns={"topic_topic_inputs" : "topic_inputs"}, inplace=True)

    topics_df = topics_df_fold.rename(columns=lambda x: x[6:]).reset_index(drop=True)
    content_df = content_df.rename(columns=lambda x: x[8:]).reset_index(drop=True)
    content_df = content_df[content_df["id"].isin(correlations["content_id"].tolist())].reset_index(drop=True)
    return topics_df, content_df, correlations_df

## valid data

In [7]:
for fold_n in cfg["general"]["fold"]:
    topics_df, content_df, correlations_df = read_data(fold_n, mode="valid")
    
    topics_en_df = topics_df[topics_df["language"]=="en"].reset_index(drop=True)
    content_en_df = content_df[content_df["language"]=="en"].reset_index(drop=True)
    topics_es_df = topics_df[topics_df["language"]=="es"].reset_index(drop=True)
    content_es_df = content_df[content_df["language"]=="es"].reset_index(drop=True)
    topics_other_df = topics_df[(topics_df["language"]!="en") & (topics_df["language"]!="es")].reset_index(drop=True)
    content_other_df = content_df[(content_df["language"]!="en") & (content_df["language"]!="es")].reset_index(drop=True)
    topics_df = pd.concat([topics_en_df, topics_es_df, topics_other_df]).reset_index(drop=True)
    content_df = pd.concat([content_en_df, content_es_df, content_other_df]).reset_index(drop=True)
    
    seed_everything(cfg["general"]["seed"], workers=True)

    # english
    model = SentenceTransformer(f"/kaggle/input/lecr-retriever-model/LECR_retriever_fold{fold_n}_model/english")
    topics_embeddings_en = model.encode(topics_en_df["topic_inputs"].tolist()) #.cpu().detach().numpy()
    content_embeddings_en = model.encode(content_en_df["content_inputs"].tolist()) #.cpu().detach().numpy()
    del model
    torch.cuda.empty_cache()
    gc.collect()

    # spanish
    model = SentenceTransformer(f"/kaggle/input/lecr-retriever-model/LECR_retriever_fold{fold_n}_model/spanish")
    topics_embeddings_es = model.encode(topics_es_df["topic_inputs"].tolist()) #.cpu().detach().numpy()
    content_embeddings_es = model.encode(content_es_df["content_inputs"].tolist()) #.cpu().detach().numpy()
    del model
    torch.cuda.empty_cache()
    gc.collect()

    # other
    model = SentenceTransformer(f"/kaggle/input/lecr-retriever-model/LECR_retriever_fold{fold_n}_model/alllang")
    topics_embeddings_other = model.encode(topics_other_df["topic_inputs"].tolist()) #.cpu().detach().numpy()
    content_embeddings_other = model.encode(content_other_df["content_inputs"].tolist()) #.cpu().detach().numpy()
    del model
    torch.cuda.empty_cache()
    gc.collect()

    topics_embeddings_en = cp.array(topics_embeddings_en)
    topics_embeddings_es = cp.array(topics_embeddings_es)
    topics_embeddings_other = cp.array(topics_embeddings_other)
    content_embeddings_en = cp.array(content_embeddings_en)
    content_embeddings_es = cp.array(content_embeddings_es)
    content_embeddings_other = cp.array(content_embeddings_other)
    
    #k_list = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 100]
    k_list = [5, 10, 50, 100]

    # english
    if len(topics_embeddings_en) != 0:
        n_neighbors = min(len(topics_embeddings_en), 1000)
        n_neighbors = n_neighbors if n_neighbors != 0 else 1
        nn = NearestNeighbors(n_neighbors, metric=cfg["task"]["metric"])
        nn.fit(content_embeddings_en)
        dists_en, indices_en = nn.kneighbors(topics_embeddings_en)
        del topics_embeddings_en, content_embeddings_en, nn
        torch.cuda.empty_cache()
        gc.collect()

    # spanish
    if len(topics_embeddings_es) != 0:
        n_neighbors = min(len(topics_embeddings_es), 1000)
        n_neighbors = n_neighbors if n_neighbors != 0 else 1
        nn = NearestNeighbors(n_neighbors, metric=cfg["task"]["metric"])
        nn.fit(content_embeddings_es)
        dists_es, indices_es = nn.kneighbors(topics_embeddings_es)
        del topics_embeddings_es, content_embeddings_es, nn
        torch.cuda.empty_cache()
        gc.collect()

    # other
    if len(topics_embeddings_other) != 0:
        n_neighbors = min(len(topics_embeddings_other), 1000)
        n_neighbors = n_neighbors if n_neighbors != 0 else 1
        nn = NearestNeighbors(n_neighbors, metric=cfg["task"]["metric"])
        nn.fit(content_embeddings_other)
        dists_other, indices_other = nn.kneighbors(topics_embeddings_other)
        del topics_embeddings_other, content_embeddings_other, nn
        torch.cuda.empty_cache()
        gc.collect()
        new_dists_other = []
        new_indices_other = []
        content_other_df["dists"] = np.nan
        for i in tqdm(topics_other_df.index):
            content_other_df.loc[indices_other[i].get(), "dists"] = dists_other[i].get()
            lang = topics_other_df.iloc[i]["language"]
            _content_df = content_other_df.iloc[indices_other[i].get()]
            _dists, _indices = _content_df[_content_df["language"]==lang]["dists"], _content_df[_content_df["language"]==lang].index
            new_dists_other.append(_dists.tolist())
            new_indices_other.append(_indices.tolist())
        content_other_df = content_other_df.drop(["dists"], axis=1)
        del _content_df, _indices, indices_other
        gc.collect()

    for k in k_list:
        print(f"[{k}]")

        # english
        predicts_en = []
        predicts_en_dists = []
        if len(topics_en_df) != 0:
            for i, d in tqdm(zip(indices_en, dists_en)):
                _content_id = content_en_df["id"].to_numpy()[i.get()].tolist()
                predicts_en.append(_content_id[:k])
                predicts_en_dists.append(d[:k].get())
        predicts_en = pd.Series(predicts_en)
        predicts_en_dists = pd.Series(predicts_en_dists)
        predicts_en = predicts_en.apply(lambda x: ' '.join(x))

        # spanish
        predicts_es = []
        predicts_es_dists = []
        if len(topics_es_df) != 0:
            for i, d in tqdm(zip(indices_es, dists_es)):
                _content_id = content_es_df["id"].to_numpy()[i.get()].tolist()
                predicts_es.append(_content_id[:k])
                predicts_es_dists.append(d[:k].get())
        predicts_es = pd.Series(predicts_es)
        predicts_es_dists = pd.Series(predicts_es_dists)
        predicts_es = predicts_es.apply(lambda x: ' '.join(x))

        # other
        predicts_other = []
        predicts_other_dists = []
        if len(topics_other_df) != 0:
            for i, d in tqdm(zip(new_indices_other, new_dists_other)):
                _content_id = content_other_df["id"].to_numpy()[i].tolist()
                predicts_other.append(_content_id[:k])
                predicts_other_dists.append(d[:k])
        predicts_other = pd.Series(predicts_other)
        predicts_other_dists = pd.Series(predicts_other_dists)
        predicts_other = predicts_other.apply(lambda x: ' '.join(x))

        # concat
        predicts = pd.concat([predicts_en, predicts_es, predicts_other]).reset_index(drop=True)
        dists = pd.concat([predicts_en_dists, predicts_es_dists, predicts_other_dists]).reset_index(drop=True)
        del predicts_en, predicts_es, predicts_other, predicts_en_dists, predicts_es_dists, predicts_other_dists
        gc.collect()
        print(predicts)

        _topics_df = topics_df.copy()
        _topics_df["predicts"] = predicts
        _topics_df["dists"] = dists
        _topics_df = _topics_df.merge(correlations_df, how="left", left_on=["id"], right_on=["topic_id"])
        _topics_df = _topics_df.dropna(subset=["topic_id", "content_ids"])
        score = f2_score(_topics_df["content_ids"], _topics_df["predicts"])
        print(score)
        pos_score = get_pos_score(_topics_df["content_ids"], _topics_df["predicts"])
        print(f"Our max positive score is {pos_score}")
        if k == 5 or k == 10 or k == 50 or k == 100:
            print("save 2nd valid data")
            content_df_copy = content_df.copy()
            content_df_copy.set_index("id", inplace = True)
            valid = build_training_set(_topics_df, content_df_copy)
            print(f"Our validation set has {len(valid)} rows")
            valid.to_pickle(f"valid_fold_k{k}_fold{fold_n}.pkl")
            print(valid.head())
            del content_df_copy, valid

        torch.cuda.empty_cache()
        gc.collect()

fold_0


/opt/conda/lib/python3.7/site-packages/pandas/core/frame.py:5047: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


Batches:   0%|          | 0/168 [00:00<?, ?it/s]

Batches:   0%|          | 0/341 [00:00<?, ?it/s]

Batches:   0%|          | 0/47 [00:00<?, ?it/s]

...

Our validation set has 378650 rows
        topic_ids     content_ids  \
0  t_0008a1bd84ba  c_6ddb1a9acadd   
1  t_0008a1bd84ba  c_7ff92a954a3d   
2  t_0008a1bd84ba  c_06571f90b3ba   
3  t_0008a1bd84ba  c_bc6425b43a74   
4  t_0008a1bd84ba  c_185375ddd0d4   

                                        topic_inputs  \
0  12. 20: Bird Reproduction [T_SEP] Libretext Op...   
1  12. 20: Bird Reproduction [T_SEP] Libretext Op...   
2  12. 20: Bird Reproduction [T_SEP] Libretext Op...   
3  12. 20: Bird Reproduction [T_SEP] Libretext Op...   
4  12. 20: Bird Reproduction [T_SEP] Libretext Op...   

                                      content_inputs  target language  \
0  12.3: Vertebrate Reproduction [C_SEP]  [C_SEP]...       0       en   
1  12. 20: Bird Reproduction [C_SEP]  [C_SEP] Is ...       1       en   
2  12. 19: Bird Structure and Function [C_SEP]  [...       0       en   
3  12. 15: Reptile Reproduction [C_SEP]  [C_SEP] ...       0       en   
4  12.2: Vertebrate Characteristics [C_S

8759it [00:00, 8818.21it/s]
605it [00:00, 7691.86it/s]
493it [00:00, 11258.31it/s]


0       c_6ddb1a9acadd c_7ff92a954a3d c_06571f90b3ba c...
1       c_c69fc19ab8f2 c_ac82abd6da1b c_e7daeadb242c c...
2       c_dde078b8ea7a c_2f241eb548e2 c_c062c17e9cba c...
3       c_f59f2a600907 c_87ad67b205d0 c_572923ccc03e c...
4       c_0d7305ef8165 c_01d56e0e228e c_9c51b0537c5b c...
                              ...                        
9852    c_74b9dc5c85ad c_8d4fd4381603 c_68b9aee7084a c...
9853    c_2eb451ff94a7 c_8cc184fbf715 c_ff90dd31104f c...
9854    c_5af91f956ee9 c_b5f3b0b26dc3 c_e903c8b2fef0 c...
9855    c_d91820d777f9 c_ce8ec8706696 c_de2d9c569e5f c...
9856    c_d45faaa9f758 c_3bcd31f4255f c_cdb915a15dc5 c...
Length: 9857, dtype: object
0.1614
Our max positive score is 0.89446
save 2nd valid data


100%|██████████| 7573/7573 [00:09<00:00, 788.39it/s]


Our validation set has 757298 rows
        topic_ids     content_ids  \
0  t_0008a1bd84ba  c_6ddb1a9acadd   
1  t_0008a1bd84ba  c_7ff92a954a3d   
2  t_0008a1bd84ba  c_06571f90b3ba   
3  t_0008a1bd84ba  c_bc6425b43a74   
4  t_0008a1bd84ba  c_185375ddd0d4   

                                        topic_inputs  \
0  12. 20: Bird Reproduction [T_SEP] Libretext Op...   
1  12. 20: Bird Reproduction [T_SEP] Libretext Op...   
2  12. 20: Bird Reproduction [T_SEP] Libretext Op...   
3  12. 20: Bird Reproduction [T_SEP] Libretext Op...   
4  12. 20: Bird Reproduction [T_SEP] Libretext Op...   

                                      content_inputs  target language  \
0  12.3: Vertebrate Reproduction [C_SEP]  [C_SEP]...       0       en   
1  12. 20: Bird Reproduction [C_SEP]  [C_SEP] Is ...       1       en   
2  12. 19: Bird Structure and Function [C_SEP]  [...       0       en   
3  12. 15: Reptile Reproduction [C_SEP]  [C_SEP] ...       0       en   
4  12.2: Vertebrate Characteristics [C_S

## train data

In [8]:
for fold_n in cfg["general"]["fold"]:
    topics_df, content_df, correlations_df = read_data(fold_n, mode="train")
    
    topics_en_df = topics_df[topics_df["language"]=="en"].reset_index(drop=True)
    content_en_df = content_df[content_df["language"]=="en"].reset_index(drop=True)
    topics_es_df = topics_df[topics_df["language"]=="es"].reset_index(drop=True)
    content_es_df = content_df[content_df["language"]=="es"].reset_index(drop=True)
    topics_other_df = topics_df[(topics_df["language"]!="en") & (topics_df["language"]!="es")].reset_index(drop=True)
    content_other_df = content_df[(content_df["language"]!="en") & (content_df["language"]!="es")].reset_index(drop=True)
    topics_df = pd.concat([topics_en_df, topics_es_df, topics_other_df]).reset_index(drop=True)
    content_df = pd.concat([content_en_df, content_es_df, content_other_df]).reset_index(drop=True)
    
    seed_everything(cfg["general"]["seed"], workers=True)

    # english
    model = SentenceTransformer(f"/kaggle/input/lecr-retriever-model/LECR_retriever_fold{fold_n}_model/english")
    topics_embeddings_en = model.encode(topics_en_df["topic_inputs"].tolist()) #.cpu().detach().numpy()
    content_embeddings_en = model.encode(content_en_df["content_inputs"].tolist()) #.cpu().detach().numpy()
    del model
    torch.cuda.empty_cache()
    gc.collect()

    # spanish
    model = SentenceTransformer(f"/kaggle/input/lecr-retriever-model/LECR_retriever_fold{fold_n}_model/spanish")
    topics_embeddings_es = model.encode(topics_es_df["topic_inputs"].tolist()) #.cpu().detach().numpy()
    content_embeddings_es = model.encode(content_es_df["content_inputs"].tolist()) #.cpu().detach().numpy()
    del model
    torch.cuda.empty_cache()
    gc.collect()

    # other
    model = SentenceTransformer(f"/kaggle/input/lecr-retriever-model/LECR_retriever_fold{fold_n}_model/alllang")
    topics_embeddings_other = model.encode(topics_other_df["topic_inputs"].tolist()) #.cpu().detach().numpy()
    content_embeddings_other = model.encode(content_other_df["content_inputs"].tolist()) #.cpu().detach().numpy()
    del model
    torch.cuda.empty_cache()
    gc.collect()

    topics_embeddings_en = cp.array(topics_embeddings_en)
    topics_embeddings_es = cp.array(topics_embeddings_es)
    topics_embeddings_other = cp.array(topics_embeddings_other)
    content_embeddings_en = cp.array(content_embeddings_en)
    content_embeddings_es = cp.array(content_embeddings_es)
    content_embeddings_other = cp.array(content_embeddings_other)
    
    #k_list = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 100]
    k_list = [5, 10, 50, 100]

    # english
    if len(topics_embeddings_en) != 0:
        n_neighbors = min(len(topics_embeddings_en), 1000)
        n_neighbors = n_neighbors if n_neighbors != 0 else 1
        nn = NearestNeighbors(n_neighbors, metric=cfg["task"]["metric"])
        nn.fit(content_embeddings_en)
        dists_en, indices_en = nn.kneighbors(topics_embeddings_en)
        del topics_embeddings_en, content_embeddings_en, nn
        torch.cuda.empty_cache()
        gc.collect()

    # spanish
    if len(topics_embeddings_es) != 0:
        n_neighbors = min(len(topics_embeddings_es), 1000)
        n_neighbors = n_neighbors if n_neighbors != 0 else 1
        nn = NearestNeighbors(n_neighbors, metric=cfg["task"]["metric"])
        nn.fit(content_embeddings_es)
        dists_es, indices_es = nn.kneighbors(topics_embeddings_es)
        del topics_embeddings_es, content_embeddings_es, nn
        torch.cuda.empty_cache()
        gc.collect()

    # other
    if len(topics_embeddings_other) != 0:
        n_neighbors = min(len(topics_embeddings_other), 1000)
        n_neighbors = n_neighbors if n_neighbors != 0 else 1
        nn = NearestNeighbors(n_neighbors, metric=cfg["task"]["metric"])
        nn.fit(content_embeddings_other)
        dists_other, indices_other = nn.kneighbors(topics_embeddings_other)
        del topics_embeddings_other, content_embeddings_other, nn
        torch.cuda.empty_cache()
        gc.collect()
        new_dists_other = []
        new_indices_other = []
        content_other_df["dists"] = np.nan
        for i in tqdm(topics_other_df.index):
            content_other_df.loc[indices_other[i].get(), "dists"] = dists_other[i].get()
            lang = topics_other_df.iloc[i]["language"]
            _content_df = content_other_df.iloc[indices_other[i].get()]
            _dists, _indices = _content_df[_content_df["language"]==lang]["dists"], _content_df[_content_df["language"]==lang].index
            new_dists_other.append(_dists.tolist())
            new_indices_other.append(_indices.tolist())
        content_other_df = content_other_df.drop(["dists"], axis=1)
        del _content_df, _indices, indices_other
        gc.collect()

    for k in k_list:
        print(f"[{k}]")

        # english
        predicts_en = []
        predicts_en_dists = []
        if len(topics_en_df) != 0:
            for i, d in tqdm(zip(indices_en, dists_en)):
                _content_id = content_en_df["id"].to_numpy()[i.get()].tolist()
                predicts_en.append(_content_id[:k])
                predicts_en_dists.append(d[:k].get())
        predicts_en = pd.Series(predicts_en)
        predicts_en_dists = pd.Series(predicts_en_dists)
        predicts_en = predicts_en.apply(lambda x: ' '.join(x))

        # spanish
        predicts_es = []
        predicts_es_dists = []
        if len(topics_es_df) != 0:
            for i, d in tqdm(zip(indices_es, dists_es)):
                _content_id = content_es_df["id"].to_numpy()[i.get()].tolist()
                predicts_es.append(_content_id[:k])
                predicts_es_dists.append(d[:k].get())
        predicts_es = pd.Series(predicts_es)
        predicts_es_dists = pd.Series(predicts_es_dists)
        predicts_es = predicts_es.apply(lambda x: ' '.join(x))

        # other
        predicts_other = []
        predicts_other_dists = []
        if len(topics_other_df) != 0:
            for i, d in tqdm(zip(new_indices_other, new_dists_other)):
                _content_id = content_other_df["id"].to_numpy()[i].tolist()
                predicts_other.append(_content_id[:k])
                predicts_other_dists.append(d[:k])
        predicts_other = pd.Series(predicts_other)
        predicts_other_dists = pd.Series(predicts_other_dists)
        predicts_other = predicts_other.apply(lambda x: ' '.join(x))

        # concat
        predicts = pd.concat([predicts_en, predicts_es, predicts_other]).reset_index(drop=True)
        dists = pd.concat([predicts_en_dists, predicts_es_dists, predicts_other_dists]).reset_index(drop=True)
        del predicts_en, predicts_es, predicts_other, predicts_en_dists, predicts_es_dists, predicts_other_dists
        gc.collect()
        print(predicts)

        _topics_df = topics_df.copy()
        _topics_df["predicts"] = predicts
        _topics_df["dists"] = dists
        _topics_df = _topics_df.merge(correlations_df, how="left", left_on=["id"], right_on=["topic_id"])
        _topics_df = _topics_df.dropna(subset=["topic_id", "content_ids"])
        score = f2_score(_topics_df["content_ids"], _topics_df["predicts"])
        print(score)
        pos_score = get_pos_score(_topics_df["content_ids"], _topics_df["predicts"])
        print(f"Our max positive score is {pos_score}")
        if k == 100:
            print("save 2nd train data")
            content_df_copy = content_df.copy()
            content_df_copy.set_index("id", inplace = True)
            train = build_training_set(_topics_df, content_df_copy)
            print(f"Our training set has {len(train)} rows")
            train.to_pickle(f"train_fold_k{k}_fold{fold_n}.pkl")
            print(train.head())
            del content_df_copy, train

        torch.cuda.empty_cache()
        gc.collect()

fold_0


/opt/conda/lib/python3.7/site-packages/pandas/core/frame.py:5047: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


Batches:   0%|          | 0/963 [00:00<?, ?it/s]

Batches:   0%|          | 0/1858 [00:00<?, ?it/s]

Batches:   0%|          | 0/389 [00:00<?, ?it/s]

...

0        c_9ca4249fa1a8 c_dbef0d3da6d3 c_65ae9a83563d c...
1        c_de6f5fbe3f97 c_447222ff490e c_f37bb15a7a42 c...
2        c_ade5a4b4e971 c_84b23a7b4b8c c_f84bee7dc0d9 c...
3        c_c1db2bde6960 c_d26e23e98356 c_b8d730238789 c...
4        c_2429552008e3 c_22eeee248488 c_699022324f4f c...
                               ...                        
67110    c_67ecb6eebadf c_140c2d4b333c c_e6b95de6962f c...
67111    c_79903740e1e8 c_e630f8201f14 c_1079bc17c6e8 c...
67112    c_4ed38ea2bff2 c_9e67dc64550d c_036efdd9e8c1 c...
67113    c_0dd3eab0f444 c_46f852a49c08 c_bb1db83147d6 c...
67114    c_e438b625e16f c_21e246a8609e c_d7f2fb1ebedf c...
Length: 67115, dtype: object
0.2624
Our max positive score is 0.94883
[100]


27402it [00:03, 8423.23it/s]
13305it [00:01, 7551.44it/s]
26408it [00:03, 7343.32it/s]


0        c_9ca4249fa1a8 c_dbef0d3da6d3 c_65ae9a83563d c...
1        c_de6f5fbe3f97 c_447222ff490e c_f37bb15a7a42 c...
2        c_ade5a4b4e971 c_84b23a7b4b8c c_f84bee7dc0d9 c...
3        c_c1db2bde6960 c_d26e23e98356 c_b8d730238789 c...
4        c_2429552008e3 c_22eeee248488 c_699022324f4f c...
                               ...                        
67110    c_67ecb6eebadf c_140c2d4b333c c_e6b95de6962f c...
67111    c_79903740e1e8 c_e630f8201f14 c_1079bc17c6e8 c...
67112    c_4ed38ea2bff2 c_9e67dc64550d c_036efdd9e8c1 c...
67113    c_0dd3eab0f444 c_46f852a49c08 c_bb1db83147d6 c...
67114    c_e438b625e16f c_21e246a8609e c_d7f2fb1ebedf c...
Length: 67115, dtype: object
0.1627
Our max positive score is 0.97785
save 2nd train data


100%|██████████| 53944/53944 [01:07<00:00, 793.56it/s]


Our training set has 5391535 rows
        topic_ids     content_ids  \
0  t_00069b63a70a  c_de6f5fbe3f97   
1  t_00069b63a70a  c_447222ff490e   
2  t_00069b63a70a  c_f37bb15a7a42   
3  t_00069b63a70a  c_c900f57fae94   
4  t_00069b63a70a  c_71d6bae3f656   

                                        topic_inputs  \
0  Transcripts [T_SEP] MIT Blossoms | Engineering...   
1  Transcripts [T_SEP] MIT Blossoms | Engineering...   
2  Transcripts [T_SEP] MIT Blossoms | Engineering...   
3  Transcripts [T_SEP] MIT Blossoms | Engineering...   
4  Transcripts [T_SEP] MIT Blossoms | Engineering...   

                                      content_inputs  target language  \
0  From Psychology to L..: Written Transcript of ...       0       en   
1  Thermodynamics: Ener..: Written Transcript of ...       0       en   
2  Quantifying the Ener..: Written Transcript of ...       0       en   
3  An Introduction to t..: Written Transcript of ...       0       en   
4  Engineering Innovati..: Written Transc